In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,4.0,2.0,13.0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,3.0,5.0,0.0,14.0,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,3.0,3.0,0.0,5.0,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,3.0,4.0,2.0,10.0,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,3.0,4.0,0.0,13.0,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [8]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [11]:
pseudo = pd.read_csv("../input/30-days-pseudo/submission.csv")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [12]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [13]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               'bootstrap_type': 'Poisson',
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 1, 3),
           'learning_rate':trial.suggest_uniform('learning_rate', 3.81e-2, 4.01e-2),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 702, 714),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 155, 163),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 30.5, 33),
            'subsample': trial.suggest_uniform('subsample', 0.97, 1)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ctb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model = ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE
       
    )


    ctb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, ctb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 15)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-23 09:32:27,528] A new study created in memory with name: no-name-22d33f7f-b301-4345-a87d-dcf7a9668380


0:	learn: 0.5936750	test: 0.7396882	best: 0.7396882 (0)	total: 13.3ms	remaining: 4m 26s
1000:	learn: 0.5699922	test: 0.7212405	best: 0.7212405 (1000)	total: 4.99s	remaining: 1m 34s
2000:	learn: 0.5652288	test: 0.7177440	best: 0.7177440 (2000)	total: 10.1s	remaining: 1m 30s
3000:	learn: 0.5621845	test: 0.7156277	best: 0.7156271 (2998)	total: 15s	remaining: 1m 25s
4000:	learn: 0.5601181	test: 0.7141786	best: 0.7141782 (3999)	total: 19.8s	remaining: 1m 19s
5000:	learn: 0.5585889	test: 0.7131613	best: 0.7131612 (4998)	total: 24.8s	remaining: 1m 14s
6000:	learn: 0.5574054	test: 0.7123903	best: 0.7123903 (6000)	total: 29.5s	remaining: 1m 8s
7000:	learn: 0.5565157	test: 0.7118855	best: 0.7118849 (6999)	total: 34.6s	remaining: 1m 4s
8000:	learn: 0.5557899	test: 0.7115065	best: 0.7115060 (7995)	total: 39.7s	remaining: 59.5s
9000:	learn: 0.5551444	test: 0.7111857	best: 0.7111857 (9000)	total: 45s	remaining: 55s
10000:	learn: 0.5546318	test: 0.7109431	best: 0.7109431 (9999)	total: 49.7s	remaining

[I 2021-08-23 09:36:43,573] Trial 0 finished with value: 0.7099791464101343 and parameters: {'max_depth': 2, 'learning_rate': 0.03992686599010366, 'max_bin': 713, 'min_data_in_leaf': 156, 'reg_lambda': 31.904112168897683, 'subsample': 0.9804663418450672}. Best is trial 0 with value: 0.7099791464101343.


0:	learn: 0.5937740	test: 0.7397706	best: 0.7397706 (0)	total: 4.58ms	remaining: 1m 31s
1000:	learn: 0.5763423	test: 0.7260146	best: 0.7260146 (1000)	total: 3.56s	remaining: 1m 7s
2000:	learn: 0.5730251	test: 0.7234051	best: 0.7234051 (2000)	total: 7.44s	remaining: 1m 6s
3000:	learn: 0.5713599	test: 0.7221322	best: 0.7221322 (3000)	total: 11s	remaining: 1m 2s
4000:	learn: 0.5701537	test: 0.7212135	best: 0.7212135 (4000)	total: 14.9s	remaining: 59.6s
5000:	learn: 0.5692035	test: 0.7205002	best: 0.7205002 (5000)	total: 19.6s	remaining: 58.7s
6000:	learn: 0.5684258	test: 0.7199279	best: 0.7199279 (6000)	total: 23.5s	remaining: 54.7s
7000:	learn: 0.5677742	test: 0.7194488	best: 0.7194488 (7000)	total: 27.6s	remaining: 51.2s
8000:	learn: 0.5672112	test: 0.7190390	best: 0.7190390 (8000)	total: 31.4s	remaining: 47.1s
9000:	learn: 0.5667178	test: 0.7186791	best: 0.7186791 (9000)	total: 34.9s	remaining: 42.7s
10000:	learn: 0.5662807	test: 0.7183671	best: 0.7183669 (9999)	total: 38.9s	remaining:

[I 2021-08-23 09:39:26,863] Trial 1 finished with value: 0.7162813067611202 and parameters: {'max_depth': 1, 'learning_rate': 0.03994305832383072, 'max_bin': 711, 'min_data_in_leaf': 156, 'reg_lambda': 32.237581866047876, 'subsample': 0.986035796530317}. Best is trial 0 with value: 0.7099791464101343.


...

0:	learn: 0.5935846	test: 0.7396151	best: 0.7396151 (0)	total: 6.58ms	remaining: 2m 11s
1000:	learn: 0.5663132	test: 0.7185930	best: 0.7185925 (999)	total: 5.52s	remaining: 1m 44s
2000:	learn: 0.5608994	test: 0.7148084	best: 0.7148080 (1999)	total: 12s	remaining: 1m 48s
3000:	learn: 0.5580790	test: 0.7130281	best: 0.7130281 (3000)	total: 17.5s	remaining: 1m 39s
4000:	learn: 0.5562558	test: 0.7119679	best: 0.7119679 (4000)	total: 23.6s	remaining: 1m 34s
5000:	learn: 0.5548805	test: 0.7112925	best: 0.7112925 (5000)	total: 29.2s	remaining: 1m 27s
6000:	learn: 0.5537993	test: 0.7108882	best: 0.7108882 (6000)	total: 35.1s	remaining: 1m 21s
7000:	learn: 0.5528914	test: 0.7106410	best: 0.7106375 (6959)	total: 40.7s	remaining: 1m 15s
8000:	learn: 0.5520866	test: 0.7104540	best: 0.7104538 (7999)	total: 47.4s	remaining: 1m 11s
9000:	learn: 0.5513444	test: 0.7103223	best: 0.7103223 (9000)	total: 53.3s	remaining: 1m 5s
10000:	learn: 0.5506628	test: 0.7102146	best: 0.7102141 (9997)	total: 59s	remai

[I 2021-08-23 10:14:47,991] Trial 13 finished with value: 0.7099422956839776 and parameters: {'max_depth': 3, 'learning_rate': 0.038102402229793116, 'max_bin': 707, 'min_data_in_leaf': 158, 'reg_lambda': 30.97064342790306, 'subsample': 0.9897232210275898}. Best is trial 11 with value: 0.7098848674925469.


0:	learn: 0.5935847	test: 0.7396147	best: 0.7396147 (0)	total: 6.74ms	remaining: 2m 14s
1000:	learn: 0.5663358	test: 0.7186078	best: 0.7186078 (1000)	total: 5.59s	remaining: 1m 46s
2000:	learn: 0.5607921	test: 0.7147368	best: 0.7147368 (2000)	total: 11.5s	remaining: 1m 43s
3000:	learn: 0.5579561	test: 0.7128835	best: 0.7128831 (2999)	total: 17.7s	remaining: 1m 40s
4000:	learn: 0.5561238	test: 0.7118395	best: 0.7118395 (4000)	total: 23.8s	remaining: 1m 35s
5000:	learn: 0.5547499	test: 0.7111854	best: 0.7111854 (5000)	total: 29.3s	remaining: 1m 28s
6000:	learn: 0.5536427	test: 0.7107694	best: 0.7107694 (6000)	total: 35s	remaining: 1m 21s
7000:	learn: 0.5527066	test: 0.7104999	best: 0.7104960 (6981)	total: 40.9s	remaining: 1m 15s
8000:	learn: 0.5518972	test: 0.7103363	best: 0.7103349 (7994)	total: 47.2s	remaining: 1m 10s
9000:	learn: 0.5511496	test: 0.7102040	best: 0.7102038 (8999)	total: 53.1s	remaining: 1m 4s
10000:	learn: 0.5504493	test: 0.7101053	best: 0.7101044 (9997)	total: 58.7s	re

[I 2021-08-23 10:17:25,645] Trial 14 finished with value: 0.7099177681639608 and parameters: {'max_depth': 3, 'learning_rate': 0.03813314986999626, 'max_bin': 707, 'min_data_in_leaf': 159, 'reg_lambda': 30.979310565754343, 'subsample': 0.9999299919229125}. Best is trial 11 with value: 0.7098848674925469.


Number of finished trials: 15
Best trial: {'max_depth': 2, 'learning_rate': 0.03816667154616651, 'max_bin': 709, 'min_data_in_leaf': 159, 'reg_lambda': 30.922021895077272, 'subsample': 0.9998995160334373}


In [15]:
study.best_params

{'max_depth': 2,
 'learning_rate': 0.03816667154616651,
 'max_bin': 709,
 'min_data_in_leaf': 159,
 'reg_lambda': 30.922021895077272,
 'subsample': 0.9998995160334373}

# Log

====== Ordinal encoding =========

0.7104169071824754 no noise ver3

0.7100365351403024 no noise ver6 (narrow space)

0.7099791926764689 no noise ver7 (narrow space)

0.7099021667949135 no noise ver8 (narrow space)

0.7098886176837581 no noise ver10 (narrow space)

0.7146235117715432 random noise ver4

0.7139039381188639 row-wise noise ver5

 ====== One-hot encoding =========
 
0.7104321496790325 no noise ver2
 

 